In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
from torch.optim import RMSprop
from datetime import datetime, timedelta

In [2]:
def transformed_time(t,T):
    return 2*(t-T)/T+1

def transformed_inventory_action(q0,q,x):
    q_hat=q/q0-1
    x_hat=x/q0
    r=np.sqrt(q_hat**2+x_hat**2)
    theta=np.arctan(-x_hat/q_hat)
    chi=-x_hat/q_hat
    if theta<=np.pi/4:
        radial_dist=r*np.sqrt((chi**2+1)*(2*np.cos(np.pi/4-theta)**2))
    else:
        radial_dist=r*np.sqrt((chi**(-2)+1)*(2*np.cos(theta-np.pi/4)**2))
    q_transform=-radial_dist*np.cos(theta)
    x_transform=radial_dist*np.sin(theta)
    return q_transform,x_transform

def transformed_price(midprice_series):
    """
    Computes the transformed price feature (P̃) from a time series of midprices.

    Parameters:
        midprice_series (pd.Series): Series indexed by timestamp (datetime), with midprice per second.

    Returns:
        pd.Series: Transformed price series (P̃), same index as input.
    """
    # Ensure the series is sorted by time
    midprice_series = midprice_series.sort_index()

    # Group by hour
    grouped = midprice_series.groupby(midprice_series.index.floor('H'))

    transformed_series = []

    for hour, group in grouped:
        # Subtract opening price of the hour
        opening_price = group.iloc[0]
        centered = group - opening_price

        # Estimate scale to fit mostly within [-1, 1]
        lower, upper = np.percentile(centered, [1, 99])  # clip only outliers
        if lower!=upper:
            # Affine transformation
            transformed = (2/(upper-lower))*(centered-upper)+1
        else:
            transformed =0*centered
        
        transformed_series.append(transformed)

    return pd.concat(transformed_series)

def QV(midprice_series):
    return np.sum(midprice_series.diff()**2)

def transformed_qv(qv,qv_mean,qv_std):
    return (qv-qv_mean)/(2*qv_std)

In [172]:
class TradingEnv:
    def __init__(self, start_date,T,N, delta_t,price_data, initial_inventory=500,a=1):
        """Time is expressed in second"""
        self.start_date=start_date
        self.T=T
        self.delta_t=delta_t #period between succesive trades in min 
        self.Tk_list = np.array([T/N*i for i in range(1,N)])
        self.Mk=T/N/delta_t
        self.price_data = price_data
        self.transormed_price=transformed_price(price_data)
        self.initial_inventory = initial_inventory

        self.current_period_index = 0
        self.inventory = initial_inventory
        self.time = self.Tk_list[0]
        self.done = False
        self.a=a

        self.state=self.get_state(self.time)

        _,self.qv_mean,self.qv_std=self.get_qv_mean_std()

    def reset(self):
        self.current_period_index = 0
        self.inventory = self.initial_inventory
        self.time = self.Tk_list[0]
        self.done = False
        return self.get_state(self.time)

    def step(self, action):
        """
        Applique l'action (ex : quantité à vendre), met à jour l'état, retourne:
        next_state, reward, done, info
        """
        #Quantiti of shares to sell
        x_Tk= action

        #Select prices of the period following the action [T_k,T_k+1[
        if self.current_period_index+1<len(self.Tk_list):
            mask=(self.start_date+timedelta(seconds=self.Tk_list[self.current_period_index])<=self.price_data.index)&(self.price_data.index<self.start_date+timedelta(seconds=self.Tk_list[self.current_period_index+1]))
        else:
            mask=(self.start_date+timedelta(seconds=self.Tk_list[self.current_period_index])<=self.price_data.index)

        selected_times=self.price_data.index[mask]
        prices=self.price_data.loc[selected_times]

        #compute reward
        reward =np.sum((self.inventory/prices.shape[0]) * prices.diff()-self.a*(x_Tk/self.Mk)**2)

        self.inventory -= x_Tk

        #Update current period
        if self.current_period_index+1<len(self.Tk_list):
            self.current_period_index += 1
            #An episode ends when all the initial inventory has been sold 
            if self.inventory <= 0:
                self.done = True
        else:
            self.done = True
            
        self.time = self.Tk_list[self.current_period_index]
        next_state = self.get_state(self.time)


        self.state=next_state
        return next_state, reward, self.done, {}
    
    def get_state(self, T_i):

        if self.current_period_index>0:
            mask=(self.start_date+timedelta(seconds=self.Tk_list[self.current_period_index-1])<=self.price_data.index)&(self.price_data.index<self.start_date+timedelta(seconds=self.Tk_list[self.current_period_index]))
        else:
            mask=(self.start_date<=self.price_data.index)&(self.price_data.index<self.start_date+timedelta(seconds=self.Tk_list[self.current_period_index]))
            
        selected_times=self.price_data.index[mask]
        prices=self.price_data.loc[selected_times]
        qv=QV(prices)

        state = [
            T_i,
            self.inventory,
            self.price_data.loc[self.start_date+timedelta(seconds=T_i):].values[0],
            qv,
        ]
        return np.array(state, dtype=np.float32)
    
    def get_transformed_state(self,state,action):

        time,q,price,qv=state

        time_tr=transformed_time(time,self.T)
        q_tr,_=transformed_inventory_action(self.initial_inventory,q,action)
        price_tr=self.transormed_price.loc[self.start_date+timedelta(seconds=int(time)):].values[0]
        qv_tr=transformed_qv(qv,self.qv_mean,self.qv_std)

        
        state = [
            time_tr,
            q_tr,
            price_tr,
            qv_tr,
        ]

        return state
    
    def get_qv_mean_std(self):

        list_qv=[]

        for current_period_index in range(len(self.Tk_list)):
            if current_period_index>0:
                mask=(self.start_date+timedelta(seconds=self.Tk_list[current_period_index-1])<=self.price_data.index)&(self.price_data.index<self.start_date+timedelta(seconds=self.Tk_list[current_period_index]))
            else:
                mask=(self.start_date<=self.price_data.index)&(self.price_data.index<self.start_date+timedelta(seconds=self.Tk_list[current_period_index]))
                
            selected_times=self.price_data.index[mask]
            prices=self.price_data.loc[selected_times]
            qv=QV(prices)
            list_qv.append(qv)

        array_qv=np.array(list_qv)
        return array_qv,np.mean(array_qv),np.std(array_qv)

In [173]:
class QNetwork(nn.Module):
    def __init__(self, input_dim):
        super(QNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(20, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(20, 20),
            nn.ReLU(),

            nn.Linear(20, 20),
            nn.ReLU(),

            nn.Linear(20, 20),
            nn.ReLU(),

            nn.Linear(20, 1)  # Q-value output
        )

    def forward(self, x):
        return self.net(x)



class TradingAgentRL:
    def __init__(self, env,state_dim, epsilon=0.1, tau=0.995, gamma=0.99, batch_size=5, memory_capacity=100, update_target_freq=10, lr=1e-3):
        self.env=env
        self.state_dim = state_dim
        self.epsilon = epsilon
        self.tau = tau
        self.gamma = gamma
        self.batch_size = batch_size
        self.update_target_freq = update_target_freq

        self.memory = deque(maxlen=memory_capacity)
        self.Q_main = QNetwork(state_dim)
        self.Q_target = QNetwork(state_dim)
        self.Q_target.load_state_dict(self.Q_main.state_dict())
        self.optimizer = RMSprop(self.Q_main.parameters(), lr)
        self.loss_fn = nn.MSELoss()

        self.iteration = 0

    def choose_action(self, state):
        q_i = state[1] 
        T_i=state[0]

        ## I we have reached terminal period [TN-1,T], we sell all the inventory 
        if T_i>=self.env.Tk_list[-1]:
            action=q_i
        else:
            if np.random.rand() < self.epsilon:
                action = np.random.binomial(q_i,1/(self.env.T-T_i))
            else:
                with torch.no_grad():
                    state_tensor = torch.FloatTensor(state).unsqueeze(0).repeat(int(q_i)+1,1)
                    actions_tensor = torch.arange(0, int(q_i) + 1).float()

                    ## Normalize inputs before using Q-network
                    transformed_states_tensor = [
                        torch.tensor(self.env.get_transformed_state(state, action), dtype=torch.float32)
                        for state, action in zip(state_tensor, actions_tensor)
                    ]
                    transformed_states_tensor=torch.stack(transformed_states_tensor)
                    inputs = torch.cat([transformed_states_tensor, actions_tensor.unsqueeze(1)], dim=1)
                    #inputs = torch.cat([state_tensor, actions_tensor.unsqueeze(1)], dim=1)
                    q_values = self.Q_main(inputs).squeeze()
                    action = torch.argmax(q_values).item()

        return action
    

    def store_transition(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))

    def train_step(self):

        # We can update Q only if we have seen enough experience (state,action,reward,next_state)
        if len(self.memory) < self.batch_size:
            return

        # We sample batch_size trasnitions from memory
        minibatch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states = zip(*minibatch)

        states = torch.FloatTensor(states)
        actions = torch.FloatTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)

        # Target computation
        with torch.no_grad():

            next_q_values=[]
            for j in range(self.batch_size):

                q_range = int(next_states[j][1])  # inventory
                ns_batch = next_states[j].unsqueeze(0).repeat(q_range + 1, 1)
                actions_batch = torch.arange(0, q_range + 1).float()
                
                ### Normalize inputs before using Q-network
                transformed_ns_batch = [
                    torch.tensor(self.env.get_transformed_state(state, action), dtype=torch.float32)
                    for state, action in zip(ns_batch, actions_batch.unsqueeze(1))
                ]
                transformed_ns_batch=torch.stack(transformed_ns_batch)
                inputs = torch.cat([transformed_ns_batch, actions_batch.unsqueeze(1)], dim=1)
                #inputs = torch.cat([ns_batch, actions_batch.unsqueeze(1)], dim=1)

                if next_states[j][0]==self.env.T:
                    next_q_value = 0
                elif next_states[j][0]==self.env.Tk_list[-1]:
                    #R(s,q)=q(p′ −p)−aq2,
                    q=states[j][1]
                    T=self.env.start_date+timedelta(seconds=self.env.T)
                    delta=self.env.delta_t
                    prices=self.env.price_data 
                    p=prices.loc[T:].values[0]
                    p_prim=prices.loc[T+timedelta(seconds=delta):].values[0]
                    next_q_value = q*(p_prim-p)-self.env.a*q**2
                else:
                    

                    #Compute Q-values with main network
                    self.Q_main.eval()
                    ### Normalize inputs before using Q-network
                    q_values = self.Q_main(inputs).squeeze()
                    ns=torch.tensor(next_states[j])
                    # Select action with highest Q-value
                    action = torch.tensor([torch.argmax(q_values).item()])

                    #Compute future Q-value with target network
                    transformed_ns = torch.tensor(self.env.get_transformed_state(ns, action), dtype=torch.float32)
                    input_target = torch.cat([transformed_ns, action]) 
                    #input_target = torch.cat([ns, action])  # shape: [5]
                    self.Q_target.eval()
                    next_q_value = self.Q_target(input_target.unsqueeze(0)).item()

                next_q_values.append(next_q_value)        
            
            next_q_values = torch.FloatTensor(next_q_values)


        targets=rewards+self.gamma*next_q_values

        ### Normalize inputs before using Q-network
        transformed_states = [
            torch.tensor(self.env.get_transformed_state(state, action), dtype=torch.float32)
            for state, action in zip(states, actions.unsqueeze(1))
        ]
        transformed_states=torch.stack(transformed_states)
        inputs = torch.cat([transformed_states ,actions.unsqueeze(1)], dim=1)
        q_preds = self.Q_main(inputs)
        
        loss = self.loss_fn(q_preds.squeeze(), targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.iteration += 1

    def train(self, num_episodes=100, N=100, update_target_every=10, tau=0.995):
        """
        Entraîne l'agent sur l'environnement `env` pendant `num_episodes` épisodes.
        À chaque épisode, effectue N pas d'interaction.
        """
        last_date=self.env.price_data.index[-1]
        dates=self.env.price_data.loc[:last_date-timedelta(seconds=self.env.T)].index
        for episode in range(num_episodes):
            # One episode is defined as an execution period of lenght T, chosen randomly in the dataset (2018 to 2023)
            self.env.start_date=random.choice(dates)
            # State is reset at the beginning of each episode
            state = self.env.reset()  
            for i in range(N-1): # N is the number of period T0<T1..<TN-1 such that an action is taken at each T_i
                
                #choose action according to epsilon-greedy policy
                action =self.choose_action(state)
                # Update current state of the environment 
                next_state, reward, done, _ = self.env.step(action)
                
                # Save transition for experience replay
                self.store_transition(state, action, reward, next_state)

                state=next_state

                # Update Q with experience replay
                self.train_step()

                if done:
                    break

            # Mise à jour du réseau cible
            if episode % update_target_every == 0:
                self.Q_target.load_state_dict(self.Q_main.state_dict())

            # Décroissance de ε
            self.epsilon = max(self.epsilon * tau, 0.01)

            print(f"Episode {episode+1}/{num_episodes} terminé, ε = {self.epsilon:.4f}")

In [174]:
data=pd.read_csv("Data/BTC_ETH_15mn.csv")
data.Date=data.Date.apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
data=data.set_index("Date")

In [175]:
env=TradingEnv(data.Price.index[0],15*500*60,100,15*60,data.Price)
agent=TradingAgentRL(env,5)

In [176]:
agent.train( num_episodes=2, N=70, update_target_every=10, tau=0.995)

/var/folders/cy/p_l7v2mn57v5jtdrn3qvp_0h0000gq/T/ipykernel_85070/3707553802.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ns=torch.tensor(next_states[j])


Episode 1/2 terminé, ε = 0.0995
Episode 2/2 terminé, ε = 0.0990


In [150]:
def pnl_agent(price_series, actions, Mk, a):
    """
    price_series : pd.Series index datetime, prix midprice à chaque tick
    actions      : list of tuples (Tk_datetime, x_Tk) – quantité à vendre au début de chaque bloc
    Mk           : nombre de pas de trading par bloc (env.Mk)
    a            : pénalité quadratique (env.a)
    """
    cash, penalty = 0.0, 0.0

    for Tk_datetime, x in actions:
        # 1) on récupère tous les prix entre Tk and Tk+1
        next_time = Tk_datetime + timedelta(seconds=env.delta_t)
        block = price_series.loc[Tk_datetime : next_time]
        if len(block)==0:
            continue

        # 2) partage linéaire sur block
        trades_per_step = x / len(block)
        cash    += (trades_per_step * block).sum()
        penalty += a * (trades_per_step ** 2) * len(block)  # ou *1 par step

    return cash - penalty

In [151]:
def pnl_twap(price_series, Q0, Mk, N, a):
    """
    TWAP: on vend Q0/N actions à chaque période Nk
    """
    # date de début
    start = price_series.index.min()
    # reconstitue la liste des Tk
    Tk_list = [start + timedelta(seconds=i * (env.T // N)) for i in range(N)]
    actions = [(Tk, Q0 / N) for Tk in Tk_list]
    return pnl_agent(price_series, actions, Mk, a)

In [152]:
def evaluate(agent, start_dates):
    """
    start_dates : liste de datetimes à tester
    Renvoie un dict {mean, median, glr, prob} des delta P&L vs TWAP en bp.
    """
    original_epsilon = agent.epsilon
    agent.epsilon = 0.0   # mode pure greedy

    deltas = []
    for sd in start_dates:
        # 1) repositionne l'env
        agent.env.start_date = sd
        state = agent.env.reset()

        # 2) collect des (Tk_datetime, x_Tk)
        actions = []
        for Tk in agent.env.Tk_list:
            action = agent.choose_action(state)
            Tk_dt = sd + timedelta(seconds=int(Tk))
            actions.append((Tk_dt, action))
            state, _, done, _ = agent.env.step(action)
            if done:
                break

        # 3) calculs P&L
        series = agent.env.price_data
        pnl_rl  = pnl_agent(series, actions, agent.env.Mk, agent.env.a)
        pnl_ref = pnl_twap(series, agent.env.initial_inventory,
                          agent.env.Mk, len(agent.env.Tk_list),
                          agent.env.a)

        # 4) delta en basis points
        deltas.append(1e4 * (pnl_rl - pnl_ref) / pnl_ref)

    agent.epsilon = original_epsilon

    arr = np.array(deltas)
    stats = {
        "mean"  : round(arr.mean(),3),
        "median": round(np.median(arr),3),
        "glr"   : round(arr[arr>0].mean() / (-arr[arr<0].mean() + 1e-8),3),
        "prob"  : round(float((arr>0).mean()*100),3)
    }
    return stats

In [177]:
from datetime import timedelta
import pandas as pd
min_time = data.index.min()
max_time = data.index.max() - timedelta(seconds=env.T)
# horaire plein toutes les T secondes
test_dates = pd.date_range(min_time, max_time, freq=f"{env.T}s")

In [178]:
stats = evaluate(agent, test_dates)
print(stats)